# Fine-tuning de Modelos de Representação para Classificação

## Baixando bibliotecas necessárias

In [8]:
!pip install evaluate==0.4.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00


## Baixando dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [3]:
train_data, valid_data, test_data = dataset["train"], dataset["validation"], dataset["test"]

## Supervised Classification

### Hugging Face Trainer

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_valid = valid_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [9]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  """Calcula F1 Score"""
  logits, label = eval_pred
  predictions = np.argmax(logits, axis=-1)
  load_f1 = evaluate.load("f1")
  f1 = load_f1.compute(predictions=predictions, references=label)["f1"]
  return {"f1":f1}


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-455917047.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Step,Training Loss
500,0.420000


TrainOutput(global_step=534, training_loss=0.41443328821703734, metrics={'train_runtime': 26.4206, 'train_samples_per_second': 322.854, 'train_steps_per_second': 20.212, 'total_flos': 227605451772240.0, 'train_loss': 0.41443328821703734, 'epoch': 1.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.32733750343322754,
 'eval_f1': 0.8619402985074627,
 'eval_runtime': 1.7561,
 'eval_samples_per_second': 607.028,
 'eval_steps_per_second': 38.153,
 'epoch': 1.0}